# Notebook 2: Text Processing & Sentiment Analysis
## FinBERT-Based Sentiment Scoring for Retail Investor Posts

---

**Research Project:** Retail Sentiment, Earnings Quality, and Stock Returns

**Purpose:** Process raw social media text and generate sentiment scores using finance-specific transformer models.

**Methodology:**
- Text cleaning preserving financial context and WSB slang
- FinBERT sentiment classification (ProsusAI/finbert)
- Daily firm-level aggregation

**Input:** `wsb_posts_raw.parquet` from Notebook 1

**Output:** Firm-day panel with sentiment and attention metrics

---

**References:**
- Araci, D. (2019). FinBERT: Financial Sentiment Analysis with Pre-trained Language Models
- Huang, A. H., Wang, H., & Yang, Y. (2023). FinBERT: A Large Language Model for Extracting Information from Financial Text

## 1. Environment Setup

In [ ]:
# =============================================================================
# INSTALL REQUIRED PACKAGES
# =============================================================================

!pip install transformers==4.35.0
!pip install torch==2.1.0
!pip install pandas==2.0.3
!pip install numpy==1.24.3
!pip install scipy==1.11.3
!pip install tqdm==4.66.1
!pip install pyarrow==14.0.1
!pip install emoji==2.8.0
!pip install nltk==3.8.1
!pip install scikit-learn==1.3.2

print("All packages installed successfully.")

In [ ]:
# =============================================================================
# IMPORT LIBRARIES
# =============================================================================

import os
import re
import json
import warnings
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional
from collections import defaultdict

import pandas as pd
import numpy as np
from scipy import stats
from tqdm.notebook import tqdm

# NLP Libraries
import torch
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    pipeline
)
import nltk
import emoji

# Download NLTK data
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print(f"\nEnvironment setup complete. Timestamp: {datetime.now()}")

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

class SentimentConfig:
    """Configuration for sentiment analysis pipeline."""
    
    # Data paths (Google Drive)
    BASE_PATH = "/content/drive/MyDrive/Research/RetailSentiment/"
    RAW_DATA_PATH = BASE_PATH + "data/raw/"
    PROCESSED_DATA_PATH = BASE_PATH + "data/processed/"
    
    # Model selection
    FINBERT_MODEL = "ProsusAI/finbert"  # Primary model
    BACKUP_MODEL = "yiyanghkust/finbert-tone"  # Alternative
    
    # Processing parameters
    MAX_SEQUENCE_LENGTH = 512  # FinBERT max tokens
    BATCH_SIZE = 32  # Adjust based on GPU memory
    
    # Text cleaning parameters
    MIN_TEXT_LENGTH = 10  # Characters
    MAX_TEXT_LENGTH = 5000  # Truncate longer texts
    
    # Aggregation windows
    TRADING_HOURS_START = 9  # EST
    TRADING_HOURS_END = 16  # EST
    
    @classmethod
    def print_config(cls):
        print("="*60)
        print("SENTIMENT ANALYSIS CONFIGURATION")
        print("="*60)
        print(f"Model: {cls.FINBERT_MODEL}")
        print(f"Max Sequence Length: {cls.MAX_SEQUENCE_LENGTH}")
        print(f"Batch Size: {cls.BATCH_SIZE}")
        print(f"Min Text Length: {cls.MIN_TEXT_LENGTH} chars")
        print("="*60)

config = SentimentConfig()
config.print_config()

In [ ]:
# =============================================================================
# MOUNT GOOGLE DRIVE
# =============================================================================

from google.colab import drive
drive.mount('/content/drive')

# Verify directories exist
os.makedirs(config.PROCESSED_DATA_PATH, exist_ok=True)
print(f"Data directories verified.")

## 2. Load Raw Data

In [ ]:
# =============================================================================
# LOAD RAW POST DATA
# =============================================================================

def load_raw_data(data_path: str) -> pd.DataFrame:
    """Load raw post data from Notebook 1.
    
    Args:
        data_path: Path to data directory
        
    Returns:
        DataFrame with raw post data
    """
    filepath = os.path.join(data_path, 'wsb_posts_raw.parquet')
    
    print(f"Loading data from: {filepath}")
    df = pd.read_parquet(filepath)
    
    print(f"\nLoaded {len(df):,} posts")
    print(f"Date range: {df['date'].min()} to {df['date'].max()}")
    print(f"\nColumns: {list(df.columns)}")
    
    return df

# Load data
raw_posts = load_raw_data(config.PROCESSED_DATA_PATH)

# Display sample
print("\nSample posts:")
raw_posts[['date', 'ticker', 'title', 'score']].head()

## 3. Text Cleaning Pipeline

### 3.1 WSB-Specific Text Cleaner

We preserve financial terminology and WSB-specific language that carries sentiment information.

In [ ]:
# =============================================================================
# TEXT CLEANING FUNCTIONS
# =============================================================================

class WSBTextCleaner:
    """Text cleaner optimized for WSB/financial social media.
    
    Design principles:
    1. Preserve sentiment-carrying elements (emojis, slang, capitalization)
    2. Remove noise (URLs, HTML, bots)
    3. Standardize format while maintaining readability
    """
    
    # WSB slang with sentiment implications
    WSB_POSITIVE_TERMS = {
        'moon', 'mooning', 'rocket', 'tendies', 'gains', 'bull', 'bullish',
        'diamond hands', 'diamondhands', 'hodl', 'hold', 'squeeze', 'gamma',
        'squeeze', 'calls', 'yolo', 'to the moon', 'stonks', 'apes', 'strong',
        'lambo', 'printing', 'brrrr', 'free money', 'cant go tits up'
    }
    
    WSB_NEGATIVE_TERMS = {
        'bear', 'bearish', 'puts', 'short', 'drill', 'drilling', 'tanking',
        'bagholding', 'bagholder', 'bag holder', 'loss', 'losses', 'red',
        'rope', 'guh', 'rekt', 'wrecked', 'worthless', 'expire worthless',
        'paper hands', 'paperhands', 'dump', 'dumping', 'crashed', 'margin call'
    }
    
    # Emoji to text mapping for sentiment preservation
    EMOJI_SENTIMENT = {
        '🚀': ' rocket bullish ',
        '🌙': ' moon bullish ',
        '💎': ' diamond hands bullish ',
        '🙌': ' hands holding ',
        '🦍': ' ape strong ',
        '📈': ' chart up bullish ',
        '📉': ' chart down bearish ',
        '💰': ' money gains ',
        '💵': ' money gains ',
        '🐂': ' bull bullish ',
        '🐻': ' bear bearish ',
        '🤡': ' clown negative ',
        '😂': ' laughing ',
        '😭': ' crying sad ',
        '🔥': ' fire hot ',
        '💀': ' dead negative ',
        '⬆️': ' up bullish ',
        '⬇️': ' down bearish ',
        '✅': ' yes positive ',
        '❌': ' no negative ',
        '🎰': ' gambling risky ',
        '🎲': ' gambling risky '
    }
    
    def __init__(self):
        # Compile regex patterns
        self.url_pattern = re.compile(r'https?://\S+|www\.\S+')
        self.html_pattern = re.compile(r'<[^>]+>')
        self.reddit_pattern = re.compile(r'\[removed\]|\[deleted\]|\[deleted by user\]')
        self.mention_pattern = re.compile(r'u/\w+|r/\w+')
        self.ticker_clean = re.compile(r'\$([A-Z]{1,5})\b')
        self.whitespace_pattern = re.compile(r'\s+')
        self.special_chars = re.compile(r'[^\w\s.,!?\'-]')
        
    def clean_text(self, text: str, preserve_emojis: bool = True) -> str:
        """Clean text while preserving sentiment information.
        
        Args:
            text: Raw text input
            preserve_emojis: Whether to convert emojis to text
            
        Returns:
            Cleaned text
        """
        if not text or not isinstance(text, str):
            return ""
        
        # Remove deleted/removed markers
        text = self.reddit_pattern.sub('', text)
        
        # Check if text is effectively empty after removal
        if len(text.strip()) < 5:
            return ""
        
        # Convert emojis to sentiment text
        if preserve_emojis:
            for emoji_char, sentiment_text in self.EMOJI_SENTIMENT.items():
                text = text.replace(emoji_char, sentiment_text)
            # Remove remaining emojis
            text = emoji.replace_emoji(text, replace=' ')
        else:
            text = emoji.replace_emoji(text, replace='')
        
        # Remove URLs
        text = self.url_pattern.sub(' ', text)
        
        # Remove HTML tags
        text = self.html_pattern.sub(' ', text)
        
        # Standardize ticker format ($TICKER -> TICKER)
        text = self.ticker_clean.sub(r'\1', text)
        
        # Remove Reddit-specific mentions
        text = self.mention_pattern.sub(' ', text)
        
        # Normalize whitespace
        text = self.whitespace_pattern.sub(' ', text)
        
        # Strip and return
        text = text.strip()
        
        return text
    
    def prepare_for_sentiment(self, text: str, max_length: int = 512) -> str:
        """Prepare text for sentiment model input.
        
        Args:
            text: Cleaned text
            max_length: Maximum character length
            
        Returns:
            Model-ready text
        """
        if not text:
            return ""
        
        # Truncate if needed (keep beginning, most relevant for posts)
        if len(text) > max_length:
            # Try to truncate at sentence boundary
            truncated = text[:max_length]
            last_period = truncated.rfind('.')
            if last_period > max_length * 0.7:
                text = truncated[:last_period + 1]
            else:
                text = truncated
        
        return text
    
    def combine_title_body(self, title: str, body: str, 
                          title_weight: int = 2) -> str:
        """Combine title and body for sentiment analysis.
        
        Title is typically more informative for sentiment, so we can
        optionally weight it more heavily.
        
        Args:
            title: Post title
            body: Post body
            title_weight: How many times to include title (1-3)
            
        Returns:
            Combined text
        """
        title = self.clean_text(title or '')
        body = self.clean_text(body or '')
        
        if title and body:
            # Repeat title for weighting
            title_repeated = ' '.join([title] * min(title_weight, 3))
            return f"{title_repeated} {body}"
        elif title:
            return title
        else:
            return body

# Initialize cleaner
text_cleaner = WSBTextCleaner()

# Test cleaning
test_texts = [
    "$GME to the moon! 🚀🚀🚀 Diamond hands baby!",
    "AAPL puts are printing. Bear gang 🐻📉",
    "Check out my DD: https://reddit.com/wsb/xxx [removed]",
    "This is gonna squeeze so hard. Not financial advice."
]

print("Text Cleaning Examples:")
print("="*60)
for text in test_texts:
    cleaned = text_cleaner.clean_text(text)
    print(f"Original: {text}")
    print(f"Cleaned:  {cleaned}")
    print("-"*60)

In [ ]:
# =============================================================================
# APPLY TEXT CLEANING TO DATASET
# =============================================================================

def clean_dataset(df: pd.DataFrame, cleaner: WSBTextCleaner) -> pd.DataFrame:
    """Apply text cleaning to entire dataset.
    
    Args:
        df: Raw posts DataFrame
        cleaner: Text cleaner instance
        
    Returns:
        DataFrame with cleaned text columns
    """
    print("Cleaning text data...")
    df = df.copy()
    
    # Clean individual columns
    tqdm.pandas(desc="Cleaning titles")
    df['title_clean'] = df['title'].progress_apply(
        lambda x: cleaner.clean_text(x) if pd.notna(x) else ''
    )
    
    tqdm.pandas(desc="Cleaning text")
    df['text_clean'] = df['text'].progress_apply(
        lambda x: cleaner.clean_text(x) if pd.notna(x) else ''
    )
    
    # Create combined text for sentiment analysis
    print("Creating combined text...")
    df['text_combined'] = df.apply(
        lambda row: cleaner.combine_title_body(
            row.get('title', ''), 
            row.get('text', '')
        ),
        axis=1
    )
    
    # Calculate text length
    df['text_length'] = df['text_combined'].str.len()
    
    # Filter out empty/very short texts
    original_count = len(df)
    df = df[df['text_length'] >= config.MIN_TEXT_LENGTH].copy()
    removed = original_count - len(df)
    
    print(f"\nCleaning complete:")
    print(f"  Original posts: {original_count:,}")
    print(f"  After cleaning: {len(df):,}")
    print(f"  Removed (too short): {removed:,}")
    print(f"  Average text length: {df['text_length'].mean():.0f} chars")
    
    return df

# Clean the dataset
cleaned_posts = clean_dataset(raw_posts, text_cleaner)

## 4. FinBERT Sentiment Model

### 4.1 Load and Initialize FinBERT

In [ ]:
# =============================================================================
# FINBERT SENTIMENT ANALYZER
# =============================================================================

class FinBERTSentimentAnalyzer:
    """Sentiment analyzer using FinBERT model.
    
    FinBERT is a BERT model pre-trained on financial text and fine-tuned
    for financial sentiment analysis.
    
    Output classes:
    - positive: Bullish/optimistic sentiment
    - negative: Bearish/pessimistic sentiment  
    - neutral: Factual/neutral content
    """
    
    def __init__(self, model_name: str = "ProsusAI/finbert", device: str = None):
        """Initialize FinBERT model.
        
        Args:
            model_name: HuggingFace model identifier
            device: 'cuda' or 'cpu'
        """
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Loading FinBERT model: {model_name}")
        print(f"Device: {self.device}")
        
        # Load tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.model.to(self.device)
        self.model.eval()
        
        # Create pipeline for easier inference
        self.pipeline = pipeline(
            "sentiment-analysis",
            model=self.model,
            tokenizer=self.tokenizer,
            device=0 if self.device == 'cuda' else -1,
            max_length=512,
            truncation=True
        )
        
        # Label mapping
        self.label_map = {
            'positive': 1,
            'negative': -1,
            'neutral': 0
        }
        
        print("Model loaded successfully.")
        
    def analyze_single(self, text: str) -> Dict:
        """Analyze sentiment of a single text.
        
        Args:
            text: Input text
            
        Returns:
            Dictionary with sentiment scores
        """
        if not text or len(text.strip()) < 5:
            return {
                'label': 'neutral',
                'score': 0.0,
                'positive': 0.33,
                'negative': 0.33,
                'neutral': 0.34,
                'polarity': 0.0
            }
        
        # Get prediction
        result = self.pipeline(text[:512])[0]
        
        # Get all class probabilities
        inputs = self.tokenizer(
            text[:512], 
            return_tensors="pt", 
            truncation=True,
            max_length=512
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            probs = torch.softmax(outputs.logits, dim=1)[0].cpu().numpy()
        
        # Map probabilities to labels
        # ProsusAI/finbert: [positive, negative, neutral]
        prob_dict = {
            'positive': float(probs[0]),
            'negative': float(probs[1]),
            'neutral': float(probs[2])
        }
        
        # Calculate polarity score: P(positive) - P(negative)
        polarity = prob_dict['positive'] - prob_dict['negative']
        
        return {
            'label': result['label'].lower(),
            'score': result['score'],
            'positive': prob_dict['positive'],
            'negative': prob_dict['negative'],
            'neutral': prob_dict['neutral'],
            'polarity': polarity
        }
    
    def analyze_batch(self, texts: List[str], batch_size: int = 32) -> List[Dict]:
        """Analyze sentiment of multiple texts in batches.
        
        Args:
            texts: List of input texts
            batch_size: Batch size for processing
            
        Returns:
            List of sentiment dictionaries
        """
        results = []
        
        # Process in batches
        for i in tqdm(range(0, len(texts), batch_size), desc="Analyzing sentiment"):
            batch = texts[i:i + batch_size]
            
            # Handle empty/short texts
            valid_indices = []
            valid_texts = []
            batch_results = [None] * len(batch)
            
            for j, text in enumerate(batch):
                if text and len(text.strip()) >= 5:
                    valid_indices.append(j)
                    valid_texts.append(text[:512])
                else:
                    batch_results[j] = {
                        'label': 'neutral',
                        'score': 0.0,
                        'positive': 0.33,
                        'negative': 0.33,
                        'neutral': 0.34,
                        'polarity': 0.0
                    }
            
            if valid_texts:
                # Tokenize batch
                inputs = self.tokenizer(
                    valid_texts,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=512
                ).to(self.device)
                
                # Get predictions
                with torch.no_grad():
                    outputs = self.model(**inputs)
                    probs = torch.softmax(outputs.logits, dim=1).cpu().numpy()
                
                # Process results
                for k, idx in enumerate(valid_indices):
                    prob = probs[k]
                    label_idx = np.argmax(prob)
                    labels = ['positive', 'negative', 'neutral']
                    
                    batch_results[idx] = {
                        'label': labels[label_idx],
                        'score': float(prob[label_idx]),
                        'positive': float(prob[0]),
                        'negative': float(prob[1]),
                        'neutral': float(prob[2]),
                        'polarity': float(prob[0] - prob[1])
                    }
            
            results.extend(batch_results)
            
            # Clear CUDA cache periodically
            if i % (batch_size * 10) == 0 and self.device == 'cuda':
                torch.cuda.empty_cache()
        
        return results

# Initialize analyzer
print("Initializing FinBERT analyzer...")
sentiment_analyzer = FinBERTSentimentAnalyzer(
    model_name=config.FINBERT_MODEL,
    device=str(device)
)

In [ ]:
# =============================================================================
# TEST SENTIMENT ANALYZER
# =============================================================================

# Test on sample texts
test_sentences = [
    "GME is going to the moon! Diamond hands!",
    "This stock is overvalued and will crash soon.",
    "Earnings report comes out tomorrow.",
    "I'm buying calls, extremely bullish on this company.",
    "Selling everything, this is going to zero.",
    "The company announced a new product launch.",
    "YOLO'd my entire savings, can't go tits up.",
    "Lost 90% of my portfolio on this garbage stock."
]

print("\nSentiment Analysis Test Results:")
print("="*80)
print(f"{'Text':<50} {'Label':<10} {'Polarity':>10}")
print("-"*80)

for text in test_sentences:
    result = sentiment_analyzer.analyze_single(text)
    print(f"{text[:48]:<50} {result['label']:<10} {result['polarity']:>10.3f}")

print("="*80)

### 4.2 Apply Sentiment Analysis to Dataset

In [ ]:
# =============================================================================
# APPLY SENTIMENT TO FULL DATASET
# =============================================================================

def apply_sentiment_analysis(
    df: pd.DataFrame,
    analyzer: FinBERTSentimentAnalyzer,
    text_column: str = 'text_combined',
    batch_size: int = 32
) -> pd.DataFrame:
    """Apply sentiment analysis to entire dataset.
    
    Args:
        df: DataFrame with text data
        analyzer: Sentiment analyzer instance
        text_column: Column containing text to analyze
        batch_size: Batch size for processing
        
    Returns:
        DataFrame with sentiment columns added
    """
    print(f"\nApplying sentiment analysis to {len(df):,} posts...")
    print(f"Using column: {text_column}")
    print(f"Batch size: {batch_size}")
    
    start_time = datetime.now()
    
    # Get texts
    texts = df[text_column].fillna('').tolist()
    
    # Analyze in batches
    results = analyzer.analyze_batch(texts, batch_size=batch_size)
    
    # Add results to DataFrame
    df = df.copy()
    df['sentiment_label'] = [r['label'] for r in results]
    df['sentiment_score'] = [r['score'] for r in results]
    df['prob_positive'] = [r['positive'] for r in results]
    df['prob_negative'] = [r['negative'] for r in results]
    df['prob_neutral'] = [r['neutral'] for r in results]
    df['sentiment_polarity'] = [r['polarity'] for r in results]
    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()
    
    print(f"\nSentiment analysis complete!")
    print(f"Duration: {duration:.1f} seconds ({len(df)/duration:.1f} posts/sec)")
    
    # Print distribution
    print(f"\nSentiment Distribution:")
    dist = df['sentiment_label'].value_counts(normalize=True)
    for label, pct in dist.items():
        print(f"  {label}: {pct*100:.1f}%")
    
    print(f"\nPolarity Statistics:")
    print(f"  Mean: {df['sentiment_polarity'].mean():.3f}")
    print(f"  Std:  {df['sentiment_polarity'].std():.3f}")
    print(f"  Min:  {df['sentiment_polarity'].min():.3f}")
    print(f"  Max:  {df['sentiment_polarity'].max():.3f}")
    
    return df

# Apply sentiment analysis
posts_with_sentiment = apply_sentiment_analysis(
    cleaned_posts,
    sentiment_analyzer,
    text_column='text_combined',
    batch_size=config.BATCH_SIZE
)

In [ ]:
# =============================================================================
# SAVE CHECKPOINT - POST-LEVEL SENTIMENT
# =============================================================================

# Save intermediate results
checkpoint_path = os.path.join(config.PROCESSED_DATA_PATH, 'wsb_posts_sentiment.parquet')
posts_with_sentiment.to_parquet(checkpoint_path, index=False)
print(f"Saved checkpoint: {checkpoint_path}")

## 5. Daily Firm-Level Aggregation

### 5.1 Aggregate to Firm-Day Panel

In [ ]:
# =============================================================================
# FIRM-DAY AGGREGATION
# =============================================================================

class FirmDayAggregator:
    """Aggregates post-level data to firm-day panel.
    
    Creates the following variables for each firm-day:
    - PostCount: Number of posts mentioning the firm
    - UniqueUsers: Distinct authors mentioning the firm
    - SentimentMean: Average polarity score
    - SentimentPosShare: Share of positive posts
    - Karma: Sum of Reddit scores
    - SentimentStd: Standard deviation of polarity (disagreement)
    """
    
    def __init__(self):
        pass
    
    def aggregate(self, df: pd.DataFrame) -> pd.DataFrame:
        """Aggregate post-level data to firm-day level.
        
        Args:
            df: Post-level DataFrame with sentiment scores
            
        Returns:
            Firm-day panel DataFrame
        """
        # Filter to posts with identified tickers
        df_with_ticker = df[df['ticker'].notna()].copy()
        print(f"Posts with tickers: {len(df_with_ticker):,}")
        
        # Ensure date is datetime
        df_with_ticker['date'] = pd.to_datetime(df_with_ticker['date'])
        
        # Create binary sentiment indicators
        df_with_ticker['is_positive'] = (df_with_ticker['sentiment_label'] == 'positive').astype(int)
        df_with_ticker['is_negative'] = (df_with_ticker['sentiment_label'] == 'negative').astype(int)
        df_with_ticker['is_neutral'] = (df_with_ticker['sentiment_label'] == 'neutral').astype(int)
        
        # Aggregation functions
        agg_funcs = {
            'platform_post_id': 'count',  # PostCount
            'author': 'nunique',  # UniqueUsers
            'sentiment_polarity': ['mean', 'std', 'median'],  # Sentiment stats
            'is_positive': 'mean',  # Share positive
            'is_negative': 'mean',  # Share negative
            'score': ['sum', 'mean'],  # Karma
            'prob_positive': 'mean',  # Average positive probability
            'prob_negative': 'mean',  # Average negative probability
            'num_comments': 'sum'  # Total comments (engagement)
        }
        
        # Group by ticker and date
        print("Aggregating to firm-day level...")
        firm_day = df_with_ticker.groupby(['ticker', 'date']).agg(agg_funcs)
        
        # Flatten column names
        firm_day.columns = [
            'PostCount',
            'UniqueUsers',
            'SentimentMean',
            'SentimentStd',
            'SentimentMedian',
            'SentimentPosShare',
            'SentimentNegShare',
            'KarmaSum',
            'KarmaMean',
            'ProbPositiveMean',
            'ProbNegativeMean',
            'TotalComments'
        ]
        
        # Reset index
        firm_day = firm_day.reset_index()
        
        # Fill NaN sentiment std with 0 (single post days)
        firm_day['SentimentStd'] = firm_day['SentimentStd'].fillna(0)
        
        # Add derived variables
        firm_day['Attention'] = np.log1p(firm_day['PostCount'])  # Log attention
        firm_day['Disagreement'] = firm_day['SentimentStd']  # Alias for clarity
        firm_day['NetSentiment'] = firm_day['SentimentPosShare'] - firm_day['SentimentNegShare']
        
        print(f"\nFirm-day panel created:")
        print(f"  Observations: {len(firm_day):,}")
        print(f"  Unique firms: {firm_day['ticker'].nunique():,}")
        print(f"  Date range: {firm_day['date'].min()} to {firm_day['date'].max()}")
        print(f"  Avg posts per firm-day: {firm_day['PostCount'].mean():.1f}")
        
        return firm_day
    
    def add_trading_day_indicators(self, firm_day: pd.DataFrame) -> pd.DataFrame:
        """Add trading day indicators for market analysis.
        
        Args:
            firm_day: Firm-day panel
            
        Returns:
            DataFrame with trading day columns
        """
        firm_day = firm_day.copy()
        
        # Day of week
        firm_day['DayOfWeek'] = firm_day['date'].dt.dayofweek
        firm_day['IsWeekend'] = firm_day['DayOfWeek'].isin([5, 6]).astype(int)
        
        # Month and year
        firm_day['Month'] = firm_day['date'].dt.month
        firm_day['Year'] = firm_day['date'].dt.year
        firm_day['YearMonth'] = firm_day['date'].dt.to_period('M')
        
        # Quarter
        firm_day['Quarter'] = firm_day['date'].dt.quarter
        
        return firm_day

# Initialize aggregator
aggregator = FirmDayAggregator()

# Create firm-day panel
firm_day_panel = aggregator.aggregate(posts_with_sentiment)
firm_day_panel = aggregator.add_trading_day_indicators(firm_day_panel)

In [ ]:
# =============================================================================
# FIRM-DAY PANEL STATISTICS
# =============================================================================

def print_panel_statistics(df: pd.DataFrame):
    """Print comprehensive statistics for firm-day panel."""
    
    print("\n" + "="*70)
    print("FIRM-DAY PANEL STATISTICS")
    print("="*70)
    
    # Basic dimensions
    print(f"\n--- Panel Dimensions ---")
    print(f"Total observations: {len(df):,}")
    print(f"Unique tickers: {df['ticker'].nunique():,}")
    print(f"Unique dates: {df['date'].nunique():,}")
    print(f"Date range: {df['date'].min()} to {df['date'].max()}")
    
    # Attention variables
    print(f"\n--- Attention Variables ---")
    print(f"Posts per firm-day:")
    print(f"  Mean: {df['PostCount'].mean():.2f}")
    print(f"  Median: {df['PostCount'].median():.1f}")
    print(f"  Max: {df['PostCount'].max():,}")
    print(f"  P95: {df['PostCount'].quantile(0.95):.0f}")
    
    print(f"\nUnique users per firm-day:")
    print(f"  Mean: {df['UniqueUsers'].mean():.2f}")
    print(f"  Median: {df['UniqueUsers'].median():.1f}")
    
    # Sentiment variables
    print(f"\n--- Sentiment Variables ---")
    print(f"Sentiment Polarity (mean):")
    print(f"  Mean: {df['SentimentMean'].mean():.4f}")
    print(f"  Std: {df['SentimentMean'].std():.4f}")
    print(f"  Min: {df['SentimentMean'].min():.4f}")
    print(f"  Max: {df['SentimentMean'].max():.4f}")
    
    print(f"\nPositive share: {df['SentimentPosShare'].mean()*100:.1f}%")
    print(f"Negative share: {df['SentimentNegShare'].mean()*100:.1f}%")
    print(f"Neutral share: {(1 - df['SentimentPosShare'].mean() - df['SentimentNegShare'].mean())*100:.1f}%")
    
    # Top tickers
    print(f"\n--- Most Discussed Tickers ---")
    top_tickers = df.groupby('ticker')['PostCount'].sum().sort_values(ascending=False).head(15)
    for ticker, count in top_tickers.items():
        print(f"  {ticker}: {count:,} posts")
    
    print("\n" + "="*70)

# Print statistics
print_panel_statistics(firm_day_panel)

### 5.2 Create Lagged and Rolling Variables

In [ ]:
# =============================================================================
# CREATE LAGGED AND ROLLING VARIABLES
# =============================================================================

def create_time_series_features(df: pd.DataFrame, 
                                windows: List[int] = [3, 5, 10, 20]) -> pd.DataFrame:
    """Create lagged and rolling window features.
    
    Args:
        df: Firm-day panel
        windows: List of rolling window sizes
        
    Returns:
        DataFrame with additional time series features
    """
    print("Creating time series features...")
    df = df.sort_values(['ticker', 'date']).copy()
    
    # Variables to create lags/rolling for
    sentiment_vars = ['SentimentMean', 'SentimentPosShare', 'PostCount', 'UniqueUsers']
    
    for var in tqdm(sentiment_vars, desc="Processing variables"):
        # Lagged values
        df[f'{var}_L1'] = df.groupby('ticker')[var].shift(1)
        df[f'{var}_L5'] = df.groupby('ticker')[var].shift(5)
        
        # Rolling means
        for window in windows:
            df[f'{var}_MA{window}'] = df.groupby('ticker')[var].transform(
                lambda x: x.rolling(window=window, min_periods=1).mean()
            )
        
        # Rolling standard deviation (for sentiment)
        if 'Sentiment' in var:
            for window in [5, 10]:
                df[f'{var}_Std{window}'] = df.groupby('ticker')[var].transform(
                    lambda x: x.rolling(window=window, min_periods=2).std()
                )
    
    # Sentiment momentum: current - MA
    df['SentimentMomentum_5'] = df['SentimentMean'] - df['SentimentMean_MA5']
    df['SentimentMomentum_10'] = df['SentimentMean'] - df['SentimentMean_MA10']
    
    # Attention surge: current vs average
    df['AttentionSurge_5'] = df['PostCount'] / df['PostCount_MA5'].replace(0, np.nan)
    df['AttentionSurge_10'] = df['PostCount'] / df['PostCount_MA10'].replace(0, np.nan)
    
    print(f"Created {len([c for c in df.columns if '_L' in c or '_MA' in c or '_Std' in c])} new features")
    
    return df

# Create time series features
firm_day_panel = create_time_series_features(firm_day_panel)

## 6. Output: Firm-Day Social Media Panel

In [ ]:
# =============================================================================
# SAVE FINAL OUTPUT
# =============================================================================

def save_firm_day_panel(df: pd.DataFrame, output_dir: str):
    """Save firm-day panel with documentation."""
    
    os.makedirs(output_dir, exist_ok=True)
    
    # Main dataset
    filepath = os.path.join(output_dir, 'wsb_firm_day_panel.parquet')
    df.to_parquet(filepath, index=False)
    print(f"Saved: {filepath}")
    
    # CSV sample
    csv_path = os.path.join(output_dir, 'wsb_firm_day_sample.csv')
    df.head(5000).to_csv(csv_path, index=False)
    print(f"Saved: {csv_path}")
    
    # Data dictionary
    data_dict = {
        'ticker': 'Stock ticker symbol',
        'date': 'Calendar date',
        'PostCount': 'Number of posts mentioning ticker on date',
        'UniqueUsers': 'Number of distinct authors posting about ticker',
        'SentimentMean': 'Average FinBERT polarity score (P(pos) - P(neg))',
        'SentimentStd': 'Standard deviation of polarity within day (disagreement)',
        'SentimentMedian': 'Median polarity score',
        'SentimentPosShare': 'Share of posts classified as positive',
        'SentimentNegShare': 'Share of posts classified as negative',
        'KarmaSum': 'Sum of Reddit scores (upvotes - downvotes)',
        'KarmaMean': 'Average Reddit score per post',
        'ProbPositiveMean': 'Average P(positive) from FinBERT',
        'ProbNegativeMean': 'Average P(negative) from FinBERT',
        'TotalComments': 'Sum of comments on posts',
        'Attention': 'Log(1 + PostCount)',
        'Disagreement': 'Alias for SentimentStd',
        'NetSentiment': 'SentimentPosShare - SentimentNegShare',
        'DayOfWeek': 'Day of week (0=Monday, 6=Sunday)',
        'IsWeekend': 'Weekend indicator',
        'Month': 'Month (1-12)',
        'Year': 'Year',
        'Quarter': 'Fiscal quarter (1-4)',
        '_L1/_L5': 'Lagged values (1 day, 5 days)',
        '_MA{n}': 'Rolling mean over n days',
        '_Std{n}': 'Rolling standard deviation over n days',
        'SentimentMomentum': 'Current sentiment minus rolling average',
        'AttentionSurge': 'Current posts / rolling average posts'
    }
    
    dict_path = os.path.join(output_dir, 'firm_day_dictionary.json')
    with open(dict_path, 'w') as f:
        json.dump(data_dict, f, indent=2)
    print(f"Saved: {dict_path}")
    
    # Summary statistics
    summary = {
        'total_observations': len(df),
        'unique_tickers': int(df['ticker'].nunique()),
        'unique_dates': int(df['date'].nunique()),
        'date_range': [str(df['date'].min()), str(df['date'].max())],
        'avg_posts_per_firmday': float(df['PostCount'].mean()),
        'avg_sentiment': float(df['SentimentMean'].mean()),
        'sentiment_std': float(df['SentimentMean'].std()),
        'model_used': config.FINBERT_MODEL,
        'created_at': datetime.now().isoformat()
    }
    
    summary_path = os.path.join(output_dir, 'firm_day_summary.json')
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2)
    print(f"Saved: {summary_path}")

# Save outputs
save_firm_day_panel(firm_day_panel, config.PROCESSED_DATA_PATH)

## 7. Summary and Next Steps

In [ ]:
# =============================================================================
# NOTEBOOK SUMMARY
# =============================================================================

print("""
╔══════════════════════════════════════════════════════════════════╗
║       NOTEBOOK 2: SENTIMENT ANALYSIS COMPLETE                    ║
╚══════════════════════════════════════════════════════════════════╝

OUTPUT FILES:
─────────────
• wsb_posts_sentiment.parquet   - Post-level data with sentiment
• wsb_firm_day_panel.parquet    - Firm-day aggregated panel
• firm_day_dictionary.json      - Variable definitions
• firm_day_summary.json         - Summary statistics

KEY VARIABLES CREATED:
──────────────────────
Attention:
  • PostCount, UniqueUsers, Attention (log), TotalComments

Sentiment:
  • SentimentMean (FinBERT polarity)
  • SentimentPosShare, SentimentNegShare
  • SentimentStd (disagreement)
  • NetSentiment

Time Series:
  • Lagged values (_L1, _L5)
  • Rolling means (_MA3, _MA5, _MA10, _MA20)
  • Momentum and surge indicators

METHODOLOGY NOTES:
──────────────────
• FinBERT model: ProsusAI/finbert
• Text cleaning preserves WSB slang and emojis
• Polarity = P(positive) - P(negative)
• Aggregation: Simple mean across posts

NEXT STEPS:
───────────
→ Notebook 3: Financial Data Collection
  - Stock prices and returns
  - Earnings announcement dates
  - Calculate earnings surprises

""")